In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2
import numpy as np
import cv2
import matplotlib.pyplot as plt
import urllib.request

In [ ]:
training_dir = 'train'
test_dir = 'test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    # rotation_range=20,
    # shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(training_dir, target_size=(48, 48), color_mode='grayscale', class_mode='categorical', batch_size=64, subset='training')
validation_generator = train_datagen.flow_from_directory(training_dir, target_size=(48, 48), color_mode='grayscale', class_mode='categorical', batch_size=64, subset='validation')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(48, 48), color_mode='grayscale', class_mode='categorical', batch_size=64)

label_map = (train_generator.class_indices)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
class myCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.95):
            print("\nReached 95% accuracy so cancelling training!")
            self.model.stop_training = True

In [ ]:
model = Sequential([
    Conv2D(32 ,(3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01)),
    Conv2D(32 ,(3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    Conv2D(64 ,(3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    # MaxPooling2D((2, 2)),
    Dropout(0.2),

    # Conv2D(128 ,(3, 3), activation='relu'),
    # BatchNormalization(),
    # # MaxPooling2D((2, 2)),
    # Dropout(0.25),

    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.2),
    # Dense(128, activation='relu'),
    # BatchNormalization(),
    # Dropout(0.4),

    Dense(7, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_318 (Conv2D)         (None, 46, 46, 32)        320       
                                                                 
 conv2d_319 (Conv2D)         (None, 44, 44, 32)        9248      
                                                                 
 batch_normalization_308 (Ba  (None, 44, 44, 32)       128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 22, 22, 32)       0         
 g2D)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 22, 22, 32)        0         
                                                                 
 conv2d_320 (Conv2D)         (None, 20, 20, 64)       

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=[myCallBack()])

In [ ]:
model.evaluate(test_generator)

In [ ]:
def url_to_image(url):
	resp = urllib.request.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
	return image

In [ ]:
classifier_dir = 'haarcascade_frontalface_default.xml'
face_classifier = cv2.CascadeClassifier(classifier_dir)

def predict(url):
    img = url_to_image(url)
    faces = face_classifier.detectMultiScale(img, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (255,0,0), 2)
        roi_img = img[y:y+h, x:x+w]
        roi_img = cv2.resize(roi_img, (48, 48), interpolation=cv2.INTER_AREA)
        roi_img = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)

        if np.sum([roi_img])!=0:
            roi = roi_img.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            preds = np.argmax(model.predict(roi))
            label= list(label_map.keys())[list(label_map.values()).index(preds)]
            label_position = (x,y)
            cv2.putText(img, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(img, 'No Face Found', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
    return img

Input the URL link of the test image

In [ ]:
url = input()
predict_img = predict(url)
plt.imshow(predict_img[:,:,::-1])